In [8]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [9]:
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec

client = MongoClient()
client = MongoClient('da1.eecs.utk.edu', 27017) 

relevancedata = []

for post in client['twitter']['LabeledStatuses_MiscRelevant_C'].find({}):  
    relevancedata.append(post) 

df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])

print(df.tags.value_counts())
df.text = df.text.astype(str)

1    6185
0    4697
Name: tags, dtype: int64


In [10]:
#DM=0
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.8124770642201836
Label Score:  [0.7770986890282919, 0.8375059592261943]


[[0.7770986890282919, 0.8375059592261943], 0.8036697247706422]

In [12]:
#lowercase and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
df.text = df.text.apply(lambda x: x.lower())
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.8077064220183485
Label Score:  [0.7819609744276707, 0.8272606391286571]


[[0.7819609744276707, 0.8272606391286571], 0.7908256880733945]

In [13]:
#Removal of Stop Words and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
#df.text = df.text.apply(lambda x: x.lower())
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.7987155963302752
Label Score:  [0.7689984233353069, 0.8206774730604846]


[[0.7689984233353069, 0.8206774730604846], 0.8018348623853211]

In [14]:
#Common word removal and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
freq = pd.Series(' '.join(df['text']).split()).value_counts()[:10]
freq = list(freq.index)
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.7908256880733944
Label Score:  [0.7461513495462828, 0.8215063920843656]


[[0.7461513495462828, 0.8215063920843656], 0.8]

In [15]:
#Rare words removal and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
freq = pd.Series(' '.join(df['text']).split()).value_counts()[-10:]
freq = list(freq.index)
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.7968807339449542
Label Score:  [0.7565958015462986, 0.8255022629345616]


[[0.7565958015462986, 0.8255022629345616], 0.7651376146788991]

In [16]:
#Spelling correction and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
from textblob import TextBlob
df['text'][:5].apply(lambda x: str(TextBlob(x).correct()))
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)


tags



Accuracy Score:  0.8078899082568807
Label Score:  [0.7746218204205318, 0.8322320409614011]


[[0.7746218204205318, 0.8322320409614011], 0.8055045871559633]

In [17]:
#Without Punctuation removal and DBOW
import pandas as pd
from pd_doc2vec import doc2vec
df = pd.DataFrame(relevancedata)
df.tags = df.tags.apply(lambda x: x[0])
df.text = df.text.astype(str)
x = doc2vec(df, "text", ["tags"])
x.score(verbose=True)



The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.
tags



Accuracy Score:  0.8012844036697248
Label Score:  [0.7659821708540002, 0.8267953554088241]


[[0.7659821708540002, 0.8267953554088241], 0.8091743119266055]